<a href="https://colab.research.google.com/github/Ebenx007/compchem-Compsci-shared-rep/blob/main/2_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from pathlib import Path
import sys
import os
import shutil
import tarfile
import zipfile
import subprocess
import pickle
import re
import glob

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# 1. RESTORE RAW DATA FROM [DATA AQUISITION NOTEBOOK](https://github.com/Ebenx007/compchem-Compsci-shared-rep/blob/main/1_data_acquisition.ipynb)
---
---

In [ ]:
#Load pickled file
def load_data(file):
 with open(file,"rb") as f:
   data = pickle.load(f)
 return(data)

In [3]:
!cp "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/raw/raw_data.tar" .

In [4]:
with tarfile.open('raw_data.tar', 'r') as f:
  f.extractall()

In [6]:
#copy raw_data and metadata into pwd 
%cp -r /content/data_acquisition/* .

In [7]:
!ls

data_acquisition			   ProgramData
drive					   programs.tar.gz
juliet_dataset_CWE_ls.pkl		   raw_data.tar
juliet_dataset_CWE_testcases_paths_ls.pkl  sample_data
juliet_dataset_ls.pkl			   submissions_ls.pkl
Juliet_Test_Suite_v1.3_for_C_Cpp.zip	   submissions_tasks_ls.pkl


# 2.  DATA PREPROCESSING 

---


##2.1 Clean Main Dataset:

*   Remove Uncompilable Source Code (platform: colab(linux))
*   Remove Non UTF8 decodable entries



In [ ]:
data_dir = '/content/ProgramData/'
count_data_entries = 0
for path, subdirs, files in os.walk(data_dir):
    for name in files:
      count_data_entries +=1
print("{} files extracted from Programming Competition Submissions archive. ".format(count_data_entries))

52000 files extracted from Programming Competition Submissions archive. 


In [ ]:
#Programming Competition Dataset Meta Data Variables   
utf8_decodable_submisisons = []
utf8_undecodable_submissions = []
compiled_cpp_code = []
compiled_c_code = []
uncompilable_utf8_decodable_submissions = []

> * Extract contents from .txt submissions, save c++ code in .cpp files, c code in .c files, and compile.

###2.1.1 Exploration:

In [ ]:
# Test extraction, conversion and compiling code with 10 samples   
cpp_distinguishing_features = ['.cpp','cin','cout']
for i in range(10):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
    except:
       pass 
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp" 
      # the -4 for the ".txt"
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      print(new_cpp_filename)

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      print(new_c_filename)



ProgramData/1/397.cpp
ProgramData/1/147.cpp
ProgramData/1/131.cpp
ProgramData/1/1076.c
ProgramData/1/2272.cpp
ProgramData/1/703.cpp
ProgramData/1/2063.c
ProgramData/1/1892.c
ProgramData/1/889.cpp
ProgramData/1/1342.cpp


In [ ]:
#Exploration 
!ls ./ProgramData/1/

In [ ]:
#Extract convert and compile the rest of the submissions
cpp_distinguishing_features = ['.cpp','cin','cout']
print('Compiling the rest of the submitted code ...')
for i in range(len(submissions_ls)):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
      utf8_decodable_submisisons.append(submissions_ls[i])
    except:
       utf8_undecodable_submissions.append(submissions_ls[i])
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp"
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      compiled_cpp_code.append(new_cpp_filename)
  

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      compiled_c_code.append(new_c_filename)
print('...' )
print('Code compilation completed')

Compiling the rest of the submitted code ...
...
Code compilation completed


In [ ]:
#Exploration
!ls ./ProgramData/104

In [ ]:
#Exploration
!ls ./ProgramData/

1    104  16  21  27  32  38  43  49  54  6   65  70  76  81  87  92  98
10   11   17  22  28  33  39  44  5   55  60  66  71  77  82  88  93  99
100  12   18  23  29  34  4   45  50  56  61  67  72  78  83  89  94
101  13   19  24  3   35  40  46  51  57  62  68  73  79  84  9   95
102  14   2   25  30  36  41  47  52  58  63  69  74  8   85  90  96
103  15   20  26  31  37  42  48  53  59  64  7   75  80  86  91  97


In [ ]:
#Exploration
print("Overview of programming Competition dataset")
print("\n Source code submissions {0}".format(len(submissions_ls)))
print("\n Utf8 Decodable.txt files directories in dataset {0}".format(len(utf8_decodable_submisisons)))
print("\n Utf8 Undecodable .txt files in dataset {0}".format(len(utf8_undecodable_submissions)))
print("\n Compiled .cpp files in dataset {0}".format(len(compiled_cpp_code)))
print("\n Compiled .c files in dataset {0}".format(len(compiled_c_code)))
print("\n Uncompilable utf8 decodable files in dataset {0}".format(len(uncompilable_utf8_decodable_submissions)))

Overview of programming Competition dataset

 Source code submissions 52000

 Utf8 Decodable.txt files directories in dataset 51752

 Utf8 Undecodable .txt files in dataset 248

 Compiled .cpp files in dataset 15406

 Compiled .c files in dataset 36594

 Uncompilable utf8 decodable files in dataset 0


In [ ]:
print('Number of Compiled C code files and visual of List')
print(len(compiled_c_code))

Number of Compiled C code files and visual of List
36594


In [ ]:
#Exploration
print(compiled_c_code)

In [ ]:
print('Number of Compiled C++ code files and visual of List')
print(len(compiled_cpp_code))

Number of Compiled C++ code files and visual of List
15406


In [ ]:
print(compiled_cpp_code)


> *  Pickle Programming Competition Dataset metadata for later use

In [ ]:
with open('submissions_ls.pkl', 'wb') as fp:
  pickle.dump(submissions_ls, fp)

In [ ]:
with open('decodable_submisisons_ls.pkl', 'wb') as fp:
  pickle.dump (utf8_decodable_submisisons, fp)

In [ ]:
with open('undecodable_submisisons_ls.pkl', 'wb') as fp:
  pickle.dump (utf8_undecodable_submissions, fp)

In [ ]:
with open('cpp_compiled_ls.pkl', 'wb') as fp:
  pickle.dump (compiled_cpp_code, fp)

In [ ]:
with open('c_compiled_ls.pkl', 'wb') as fp:
  pickle.dump (compiled_c_code, fp)

##2.2 Clean New Dataset:

*   Remove Uncompilable Source Code (platform: colab(linux))

> *   Extract files from Juliet C/C++ test suite 

In [ ]:
with zipfile.ZipFile('Juliet_Test_Suite_v1.3_for_C_Cpp.zip', 'r') as juliet_dataset:
   juliet_dataset.extractall()
   print('Done Extracting dataset')

Done Extracting dataset


In [ ]:
#View of extracted Juliet_dataset directory
!ls /content/C

compile_all.bat		 py_common.py
create_per_cwe_files.py  run_analysis_example_tool.py
doc			 testcases
Makefile		 testcasesupport
manifest.xml		 update_main_cpp_and_testcases_h.py


In [ ]:
#Check number of code files i.e. size of dataset
juliet_code_files_dir = '/content/C/testcases/'
test_file_counter = 0
for path, subdirs, files in os.walk(juliet_code_files_dir):
    for name in files:
      test_file_counter +=1
print("{} files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files".format(test_file_counter))

106075 files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files


In [ ]:
#Compilating CWE IDed compilable/makeFile-containing CWE IDed directories 
CWE_makefile_dir = []
cwd = os.getcwd()
print("Compiling ...")
for i in range(len(juliet_dataset_ls)):
  if juliet_dataset_ls[i].endswith('Makefile') and 'CWE' in juliet_dataset_ls[i]:
    CWE_makefile_dir.append(juliet_dataset_ls[i])
    redx = re.findall(r'\bCWE\w.*', juliet_dataset_ls[i])
    os.chdir(os.path.dirname(juliet_dataset_ls[i])) 
    print(redx[0][:-8]) 
    #Minus 8 for the length of the word Makefile leading to Makefile containing directories which are the only linux compilables
    with open(os.path.join(cwd, os.path.dirname(juliet_dataset_ls[i]),'compilation.log'), 'w') as lg:
      subprocess.run(["make", "stdout=lg"])
    os.chdir(cwd)

#152 individual CWE IDed dir with Makefiles
print("Compilated all {} Makefile containing CWE IDed directories".format(len(CWE_makefile_dir)))
print('\n')

Compiling ...
CWE789_Uncontrolled_Mem_Alloc/s01/
CWE789_Uncontrolled_Mem_Alloc/s02/
CWE484_Omitted_Break_Statement_in_Switch/
CWE194_Unexpected_Sign_Extension/s01/
CWE194_Unexpected_Sign_Extension/s02/
CWE563_Unused_Variable/
CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/
CWE78_OS_Command_Injection/s03/
CWE78_OS_Command_Injection/s01/
CWE78_OS_Command_Injection/s04/
CWE78_OS_Command_Injection/s02/
CWE390_Error_Without_Action/
CWE364_Signal_Handler_Race_Condition/
CWE188_Reliance_on_Data_Memory_Layout/
CWE127_Buffer_Underread/s03/
CWE127_Buffer_Underread/s01/
CWE127_Buffer_Underread/s02/
CWE775_Missing_Release_of_File_Descriptor_or_Handle/
CWE667_Improper_Locking/
CWE843_Type_Confusion/
CWE587_Assignment_of_Fixed_Address_to_Pointer/
CWE480_Use_of_Incorrect_Operator/
CWE762_Mismatched_Memory_Management_Routines/s03/
CWE762_Mismatched_Memory_Management_Routines/s07/
CWE762_Mismatched_Memory_Management_Routines/s01/
CWE762_Mismatched_Memory_Management_Routines/s05/
CWE762_Mismatc

In [ ]:
#Create list of all compiled juliet code along with c and cpp compiled code for easy parsing with appropriate tools

compilable_juliet_code = []
compilable_juliet_c_code = []
compilable_juliet_cpp_code = []
paths = Path('./C/testcases').glob('**/*.o')
for path in paths:
  if re.search(r'CWE',str(path.name)):
    #.name to make sure the last part of the name i.e. before the suffix is verified
    compilable_juliet_code.append(str(path))
    if Path(str(path)[:-2]+'.c').exists():
      #Back to Path allows you to use the .exists atribute, while str allows -2 for ".o" and + operator for +'.c'
      compilable_juliet_c_code.append(str(path)[:-2]+'.c')
    if Path(str(path)[:-2]+'.cpp').exists():
      compilable_juliet_cpp_code.append(str(path)[:-2]+'.cpp')
 
  # str because path is an object not string
print("Total of {} compilable code:".format(len(compilable_juliet_code)))
print(compilable_juliet_code[0])
print("\n{} of the compilable Juliet code is C:".format(len(compilable_juliet_c_code)))
print(compilable_juliet_c_code[0])
print("\n{} of the compilable Juliet code is C++:".format(len(compilable_juliet_cpp_code)))
print(compilable_juliet_cpp_code[0])
print('\n35551 + 30441 = {} matching the total compilable code in the Juliet dataset'.format(35551 + 30441))
print('\n')

Total of 65992 compilable code:
C/testcases/CWE563_Unused_Variable/CWE563_Unused_Variable__unused_value_long_82_goodG2B.o

35551 of the compilable Juliet code is C:
C/testcases/CWE563_Unused_Variable/CWE563_Unused_Variable__unused_init_variable_long_05.c

30441 of the compilable Juliet code is C++:
C/testcases/CWE563_Unused_Variable/CWE563_Unused_Variable__unused_value_long_82_goodG2B.cpp

35551 + 30441 = 65992 matching the total compilable code in the Juliet dataset




###2.2.1 Exploration:

In [ ]:
print('Number of labels/CWE IDed Makefiles is {}'.format(len(CWE_makefile_dir)))
print('CWE IDed Makefiles is :')

Number of labels/CWE IDed Makefiles is 152
CWE IDed Makefiles is :


In [ ]:
print(CWE_makefile_dir)

In [ ]:
#Get juliet list from file back into python variable for further processing 
juliet_dataset_ls = load_data('juliet_dataset_ls.pkl')

In [ ]:
with open('compilable_juliet_code_ls.pkl', 'wb') as fp:
  pickle.dump (compilable_juliet_code, fp)

In [ ]:
with open('compilable_juliet_c_code_ls.pkl', 'wb') as fp:
  pickle.dump (compilable_juliet_c_code, fp)

In [ ]:
with open('compilable_juliet_cpp_code_ls.pkl', 'wb') as fp:
  pickle.dump (compilable_juliet_cpp_code, fp)

In [ ]:
with open('CWE_makefile_dir_ls.pkl', 'wb') as fp:
  pickle.dump (CWE_makefile_dir, fp)

# 3.  PREPROCESS SERIALIZE & STORE DATA 
---
>  *  Use pycparser to extract ASTs from C code

##3.0 Clone Pycparser Repository to ease access to its fake headers and c_json functions 


In [ ]:
!pip list -v | grep [Pp]ycparser

pycparser                     2.21           /usr/local/lib/python3.7/dist-packages pip


In [ ]:
#see the files in a package 
!pip3 show -f pycparser

In [ ]:
#Clone pycparser Repository  
!git clone  https://github.com/eliben/pycparser.git 

Cloning into 'pycparser'...
remote: Enumerating objects: 2838, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 2838 (delta 221), reused 165 (delta 82), pack-reused 2490
Receiving objects: 100% (2838/2838), 1.24 MiB | 15.09 MiB/s, done.
Resolving deltas: 100% (1898/1898), done.


In [ ]:
!ls ./pycparser/examples/

cdecl.py   dump_ast.py		   func_defs.py    serialize_ast.py
c_files    explore_ast.py	   __pycache__	   using_cpp_libc.py
c_json.py  func_calls.py	   README.rst	   using_gcc_E_libc.py
c-to-c.py  func_defs_add_param.py  rewrite_ast.py


In [ ]:
root = os.getcwd()
sys.path.append(root + "/pycparser/examples/")
from c_json import to_dict, file_to_dict, file_to_json

##3.1 Serialize Main Dataset :
Pedagogical Programming Open Judge (OJ) system C/C++ Programming Competition Data


In [ ]:
#Preprocessing Programming Competition Dataset for pycparser into *_pp.c
preprocessed_main_dataset_ls = []
print('Preprocessing main dataset Compilable c code into *_pp.c for pycparsing ...')
for i in range(len(c_compiled_ls)):
  input_flags = ['-nostdinc', '-E', '-Ipycparser/utils/fake_libc_include', c_compiled_ls[i]]
  my_cmd = ['gcc'] + input_flags
  pp_filename = c_compiled_ls[i][:-2] + '_pp.c'
  with open(os.path.join('',pp_filename ), 'w') as outfile:
    subprocess.run(my_cmd, stdout= outfile)
  preprocessed_main_dataset_ls.append(pp_filename)
print('...' )
print('Code preprocessing complete')

Preprocessing main dataset Compilable c code into *_pp.c for pycparsing ...
...
Code preprocessing complete


In [ ]:
#Parse preprocessed Programming Competition main dataset with Pycparser, and store as file_path, ast-dicts and json AST_tuples  
main_dataset_json_asts_tup_ls = []
for i in range(len(preprocessed_main_dataset_ls)):
  data_file_path = preprocessed_main_dataset_ls[i]
  ast_dict = file_to_dict(data_file_path)
  ast_json = file_to_json(data_file_path)
  main_dataset_json_asts_tup_ls.append((data_file_path, ast_dict, ast_json)) 

In [ ]:
print(len(main_dataset_json_asts_tup_ls))

36594


In [ ]:
#Explore
main_dataset_json_asts_tup_ls[0:1]

In [ ]:
with open('data_ast__path_dict_json__tup_ls.pkl', 'wb') as fp:
  pickle.dump (main_dataset_json_asts_tup_ls, fp)

In [ ]:
#create dir to store preprocessed_main_data AST tuples for archiving
!mkdir data_path_ast_dicts_jsons_tups_ls

In [ ]:
%cp -r data_ast__path_dict_json__tup_ls.pkl data_path_ast_dicts_jsons_tups_ls/

In [ ]:
shutil.make_archive('preprocessed_main_data','tar','/content/','data_path_ast_dicts_jsons_tups_ls')

'/content/preprocessed_main_data.tar'

In [ ]:
!cp preprocessed_main_data.tar "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/interim/"

##3.2 Serialize New Dataset :
National Institute of Standards and Technology(NIST)'s Software Assurance Reference Dataset (SARD)--Juliet C/C++ test suite.


In [ ]:
#Preprocessing the NIST (SARD)--Juliet C/C++ test suite data for pycparser into *_pp.c
gcc_preprocessed_nist_juliet_dataset_ls = []
print('Preprocessing new dataset Compilable c code into *_pp.c for pycparsing ...')
for i in range(len(compilable_juliet_c_code_ls)):
  input_flags = ['-nostdinc', '-E', '-IC/testcasesupport/', '-Ipycparser/utils/fake_libc_include', compilable_juliet_c_code_ls[i]]
  my_cmd = ['gcc'] + input_flags
  pp_filename = compilable_juliet_c_code_ls[i][:-2] + '_pp.c'
  with open(os.path.join('',pp_filename ), 'w') as outfile:
    subprocess.run(my_cmd, stdout= outfile)
  gcc_preprocessed_nist_juliet_dataset_ls.append(pp_filename)
print('...' )
print('Code preprocessing complete')

Preprocessing new dataset Compilable c code into *_pp.c for pycparsing ...
...
Code preprocessing complete


For effective RAM use we split NIST dataset into mini-batches of 10K each:


1.   mini-batch 1 (juliet_dataset__path_ast_json_1__tup_ls):(0,10000)
2.   mini-batch 2 (juliet_dataset__path_ast_json_2__tup_ls):(10000, 20000)
3.   mini-batch 3 ((juliet_dataset__path_ast_json_3__tup_ls):(20000, 30000)
4.   mini-batch 4 ((juliet_dataset__path_ast_json_4__tup_ls):(30,000, len(gcc_preprocessed_nist_juliet_dataset_ls))





In [ ]:
#Parse 4th and last mini-batch of preprocessed NIST (SARD)--Juliet C/C++ test suite data wit Pycparser and stored as file_path, ast_dicts, and json ASTs tuples 
juliet_dataset__path_ast_json_4__tup_ls = []
for i in range(30000, len(gcc_preprocessed_nist_juliet_dataset_ls)):
  data_file_path = gcc_preprocessed_nist_juliet_dataset_ls[i]
  ast_dict = file_to_dict(data_file_path)
  ast_json = file_to_json(data_file_path)
  juliet_dataset__path_ast_json_4__tup_ls.append((data_file_path, ast_dict, ast_json)) 

In [ ]:
juliet_dataset__path_ast_json_4__tup_ls[0]

In [ ]:
!mkdir juliet_mini_batch_4

In [ ]:
with open('juliet_dataset__path_ast_json_4__tup_ls.pkl', 'wb') as fp:
  pickle.dump (juliet_dataset__path_ast_json_4__tup_ls, fp)

In [ ]:
#Move preprocessed main data and pickled meta data to folder for compression and storage for use in next stage of processing 
%cp -r juliet_dataset__path_ast_json_4__tup_ls.pkl juliet_mini_batch_4/

In [ ]:
shutil.make_archive('juliet_mini_batch_4__path_ast_json__tup_ls','tar','/content/','juliet_mini_batch_4')

'/content/juliet_mini_batch_4__path_ast_json__tup_ls.tar'

In [ ]:
!cp juliet_mini_batch_4__path_ast_json__tup_ls.tar "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/interim/"